In [ ]:
# import sys
# !{sys.executable} -m pip install --extra-index-url https://artifactory.vgt.vito.be/api/pypi/python-packages/simple terracatalogueclient

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import os, requests, json, geojson
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, shape
from shapely.ops import unary_union
import rasterio
from rasterio.features import shapes
from shapely.ops import transform
from rasterio.merge import merge as rmerge
import rasterio.mask
import fiona
import shapely
import pyproj
import boto3
from terracatalogueclient import Catalogue as Terracat

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"
boundary_ext = '/data/boundaries/'
indicators_file_aws = 'indicators/indicators.csv'

In [ ]:
OUTPUT_FILENAME = 'LND-8-builtup-KBAs3.csv'

In [ ]:
KBA_FILENAME = 'data/KBA_data/KBAsGlobal_2022_March_01_POL'
KBA_TEMPFILENAME = 'temp_kbadata'

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
project = pyproj.Transformer.from_crs(
    pyproj.CRS.from_epsg(4326), # source coordinate system
    pyproj.CRS.from_epsg(4326), # destination coordinate system
    always_xy=True
)
# Get ESA Wo

In [ ]:
worldcover_catalogue = Terracat().authenticate() 
terracat = worldcover_catalogue.authenticate_non_interactive('tedwongwri', 'WRIpass12!')

In [ ]:
source_bucket = s3.Bucket(bucket_name)
for ext in ('shp', 'shx'):
    source_bucket.download_file('{0}.{1}'.format(KBA_FILENAME, ext), '{0}.{1}'.format(KBA_TEMPFILENAME, ext))

kba_multi = shapely.geometry.MultiPolygon([shapely.geometry.shape(poly['geometry']) for poly in fiona.open('{0}.shp'.format(KBA_TEMPFILENAME))]) 

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + boundary_ext + 'boundary_georef.csv')
boundary_georef

In [ ]:
def boundingbox_wkt(p):
    # Returns WKT for bounding box.
    # Necessary because GBIF API won't accept complex polygons.
    minx, miny, maxx, maxy = p.bounds
    return 'POLYGON (({0} {3}, {0} {2}, {1} {2}, {0} {3}))'.format(str(minx), str(maxx), str(miny), str(maxy))

def classify_builtup(r):
    r[r != 50] = 0
    r[r == 50] = 1    # built up
    return r

def mask_raster_with_geometry(raster, transform, shapes, **kwargs):
    # This function clips and masks raster
    # Adapted from https://gis.stackexchange.com/a/387772
    """Wrapper for rasterio.mask.mask to allow for in-memory processing.

    Docs: https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html

    Args:
        raster (numpy.ndarray): raster to be masked with dim: [H, W]
        transform (affine.Affine): the transform of the raster
        shapes, **kwargs: passed to rasterio.mask.mask

    Returns:
        masked: numpy.ndarray or numpy.ma.MaskedArray with dim: [H, W], and new affine transform
    """
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(
            driver='GTiff',
            height=raster.shape[0],
            width=raster.shape[1],
            count=1,
            dtype=raster.dtype,
            transform=transform,
        ) as dataset:
            dataset.write(raster, 1)
        with memfile.open() as dataset:
            output, new_transform = rasterio.mask.mask(dataset, shapes, **kwargs)
    return output.squeeze(0), new_transform

def do_one_geom(row):
    poly = row[0]
    shapelybox = poly.bounds
    kba_box = shapely.ops.clip_by_rect(kba_multi, *shapelybox)
    kba_box = shapely.ops.unary_union(kba_box)
    kba_poly = kba_box.intersection(poly)
    kbapoly_area = kba_poly.area
    if kbapoly_area > 0:
    
        box = boundingbox_wkt(poly)

        products = terracat.get_products('urn:eop:VITO:ESA_WorldCover_10m_2020_V1', geometry=box)

        # Create mosaic raster of Worldcover rasters downloaded from ESA
        rasters_to_mosaic = []
        for product in products:
            if not '{0}dir'.format(product.title) in os.listdir('.'):
                terracat.download_file(product.data[0], '{0}dir'.format(product.title))
            raster = rasterio.open('{0}dir/{0}_Map.tif'.format(product.title))
            rasters_to_mosaic.append(raster)

        mosaic, mosaic_transform = rmerge(rasters_to_mosaic)


        # Clip raster to district boundary
        if poly.type == 'Polygon':
            multipoly = MultiPolygon([poly])
        else:
            multipoly = poly

        clipped_raster, clip_transform = mask_raster_with_geometry(mosaic[0], mosaic_transform, multipoly, crop=True)
        # Classify clipped raster as habitat/nonhabitat
        builtup_raster = classify_builtup(clipped_raster)


        # Vectorize and collect only the builtup patches
        with rasterio.Env():
            image = builtup_raster
            results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for (s, v) 
            in shapes(image, transform=clip_transform) if v == 1)
        shapelist = list(results)
        shapelist = shapelist[:-1]
        all_patches = [  # project to UTM so that spatial unit is meter
            transform(project.transform, Polygon(shapelist[i]['geometry']['coordinates'][0])) for i in range(len(shapelist))
        ]
        patches = [j.simplify(0, preserve_topology=False) for j in all_patches]
        #patchgeoms = gpd.GeoDataFrame(geometry=patches, crs='EPSG:{}'.format(target_epsg), index=range(len(patches)))
        #patchgeoms_sindex = patchgeoms.sindex

        builtup_multi = MultiPolygon(patches)

        builtup_box = shapely.ops.clip_by_rect(builtup_multi, *shapelybox)
        builtup_box = shapely.ops.unary_union(builtup_box)    # unary_union repairs inalid geometries

        builtup_kba_poly = builtup_box.intersection(kba_poly)

        result = builtup_kba_poly.area/kbapoly_area
    else:
        result = -9999

    print('{0}: {1}'.format(row[1], result))
    return result

In [ ]:
for i in range(16,17): #0,len(boundary_georef)): # cities not working: 16
    if not OUTPUT_FILENAME in os.listdir('.'):
        so_far_df = pd.DataFrame()
        so_far_df.to_csv(OUTPUT_FILENAME)
        so_far = []
    else:
        so_far_df = pd.read_csv(OUTPUT_FILENAME)
        so_far = [so_far_df.iloc[j]['geo_id'] for j in range(len(so_far_df))]
    
    most_recent = []
    #outputs_df = pd.DataFrame()
    for boundary_name in ['aoi_boundary_name', 'units_boundary_name']:
        if type(boundary_georef.loc[i, boundary_name]) != float: # sometimes boundary_id is nan
            boundary_id = boundary_georef.loc[i, 'geo_name']+'-' + boundary_georef.loc[i, boundary_name]
            boundary_path = aws_s3_dir + boundary_ext +'boundary-'+boundary_id+'.geojson'
            boundary_geo = requests.get(boundary_path).json()
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
            temp_gdf['LND_8_percentKBAsBuiltup'] = temp_gdf.apply(do_one_geom, axis=1)
            most_recent.append(temp_gdf.copy())
            
    # Delete WorldCover rasters to prevent filling up disk
    for directoryname in os.listdir('.'):
        if directoryname[:15] == 'ESA_WorldCover_':
            for filename in os.listdir(directoryname):
                os.remove('{0}/{1}'.format(directoryname, filename))
            os.rmdir(directoryname)
            
    result = pd.concat([so_far_df] + most_recent, axis=0)
    result[['geometry', 'geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'LND_8_percentKBAsBuiltup']].to_csv(OUTPUT_FILENAME)

In [ ]:
processedcities = pd.read_csv(OUTPUT_FILENAME)
processedcities

In [ ]:
csv1 = pd.read_csv('LND-8-builtup-KBAs.csv')
csv2 = pd.read_csv('LND-8-builtup-KBAs3.csv')
csv3 = pd.read_csv('LND-8-builtup-KBAs2.csv')

processedcities = csv1.append(csv2).append(csv3)
processedcities

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir +'/'+ indicators_file_aws)
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = processedcities,
                                            indicator_name = 'LND_8_percentKBAsBuiltup')

In [ ]:
cities_indicators_merged

## Upload in aws s3

In [ ]:
# upload to aws
key_data = indicators_file_aws
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')

In [ ]:
# delete temporary files
for ext in ('shp', 'shx'):
    os.remove('{0}.{1}'.format(KBA_TEMPFILENAME, ext))